# **Baixador de Dados GOES16 AWS e Geração de Imagens**

In [ ]:
from google.colab import drive
#-- Montando o Drive
drive.mount('/content/drive')

In [ ]:
# Installing the numpy, netcdf4, boto3 and gdal libraries
!pip install -q "numpy<2.0" xarray numpy cartopy boto3 gdal salem rasterio pyproj geopandas descartes metpy
print ('\n')

In [ ]:
# Download the "utilities.py" script
!wget -c https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/utils/utilities.py
print('\n')

# Download a "CPT" file for IR channels
!wget -c https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/utils/IR4AVHRR6.cpt

In [ ]:
!pip install numpy --upgrade --force-reinstall
import os
os.kill(os.getpid(), 9)

In [ ]:
#@title Imagens GOES16 Canais 02, 06, 07 e 13 com Estrelas
import os
import boto3
from botocore.client import Config
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import numpy as np
from utilities import download_CMI, remap, loadCPT

# Diretórios
input_dir = "./samples"; os.makedirs(input_dir, exist_ok=True)
output_dir = "./output"; os.makedirs(output_dir, exist_ok=True)

# Intervalo de datas
start_date = datetime(2024, 8, 15)
end_date = datetime(2024, 9, 10)
bands = [2,13]
hour = 18
extent = [-90.00, -60.00, -30.00, 15.00]

# Títulos por canal
band_titles = {
    2: "Canal 2 (0.64 µm) - Visível",
#    6: "Canal 6 (2.2 µm) - SWIR",
#    7: "Canal 7 (3.9 µm) - SWIR",
    13: "Canal 13 (10.3 µm) - IR realçado"
}

# Coordenadas das estações e cores das estrelas
estacoes = {
    'São_Paulo': {'coords': (-23.5505, -46.6333), 'cor': 'white'},
    'Cuiaba-Miranda': {'coords': (-15.73091, -56.07086), 'cor': 'darkred'},
    'Alta-Floresta': {'coords': (-9.908354, -56.064393), 'cor': 'darkgreen'}
}

# Loop para baixar e processar
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    for band in bands:
        yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
        print(f'\n>>> {yyyymmdd} - Canal {band}')
        try:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1: continue

            path = f'{input_dir}/{file_name}.nc'
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            # Configuração por banda
            if band == 2:
                vmin, vmax = 0.0, 1.0
                label = 'Refletância (canal visível)'
                cmap = cm.gray
            elif band == 6:
                vmin, vmax = 0.0, 0.5
                label = 'Radiância (SWIR - 2.2 µm)'
                cmap = cm.cubehelix
            elif band == 7:
                data = data - 273.15
                vmin, vmax = 0.0, 60
                label = 'Radiância (SWIR - 3.9 µm)'
                cmap = cm.plasma
            elif band == 13:
                data = data - 273.15
                vmin, vmax = -80, 30
                label = 'Temperatura de brilho (°C)'
                cmap = loadCPT('IR4AVHRR6.cpt')

            colormap = cm.colors.LinearSegmentedColormap('cpt', cmap) if isinstance(cmap, dict) else cmap

            # Figura
            plt.figure(figsize=(10,10))
            ax = plt.axes(projection=ccrs.PlateCarree())
            img_extent = [extent[0], extent[2], extent[1], extent[3]]
            img = ax.imshow(data, origin='upper', extent=img_extent, vmin=vmin, vmax=vmax, cmap=colormap)

            ax.coastlines(resolution='10m', color='white', linewidth=0.8)
            ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
            gl = ax.gridlines(draw_labels=True, xlocs=np.arange(-180,180,5), ylocs=np.arange(-90,90,5),
                              linestyle='--', linewidth=0.25, color='gray', alpha=1.0)
            gl.top_labels = gl.right_labels = False

            shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
            ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

            # Adicionando as estrelas e os nomes
            for nome, info in estacoes.items():
                lat, lon = info['coords']
                cor = info['cor']
                # Estrela colorida com contorno preto
                ax.plot(lon, lat, marker='*', color=cor, markersize=15,
                        markeredgecolor='black', markeredgewidth=1.5,
                        transform=ccrs.PlateCarree())
                # Nome sempre preto
                ax.text(lon + 1, lat + 1, nome.replace('-', ' '), color='white',
                        fontsize=10, transform=ccrs.PlateCarree())

            # Tempo
            date = datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            date_str = date.strftime('%Y-%m-%d %H:%M UTC')
            title_band = f"GOES-16 {band_titles[band]}"

            plt.colorbar(img, label=label, extend='both', orientation='horizontal', pad=0.05, fraction=0.05)
            plt.title(f'{title_band}\n{date_str}', fontweight='bold', fontsize=10, loc='left')
            plt.title(f'Reg.: {extent}', fontsize=10, loc='right')

            out_file = f'{output_dir}/GOES16_B{band}_{yyyymmddhhmn}.png'
            plt.savefig(out_file, bbox_inches='tight', transparent=True, pad_inches=0.1, dpi=300)
            plt.close()
            print(f'[✓] Imagem salva: {out_file}')
        except Exception as e:
            print(f'[!] Erro para {yyyymmdd} B{band}: {e}')
    current_date += timedelta(days=1)

In [ ]:
#@title Imagens GOES16 Canais 02, 06, 07 e 13
import os
import boto3
from botocore.client import Config
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import numpy as np
from utilities import download_CMI, remap, loadCPT

# Diretórios
input_dir = "./samples"; os.makedirs(input_dir, exist_ok=True)
output_dir = "./output"; os.makedirs(output_dir, exist_ok=True)

# Intervalo de datas
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
bands = [2, 6, 7, 13]
hour = 18
extent = [-90.00, -60.00, -30.00, 15.00]

# Títulos por canal
band_titles = {
    2: "Canal 2 (0.64 µm) - Visível",
    6: "Canal 6 (2.2 µm) - SWIR",
    7: "Canal 7 (3.9 µm) - SWIR",
    13: "Canal 13 (10.3 µm) - IR realçado"
}

# Loop para baixar e processar
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    for band in bands:
        yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
        print(f'\n>>> {yyyymmdd} - Canal {band}')
        try:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1: continue

            path = f'{input_dir}/{file_name}.nc'
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            # Configuração por banda
            if band == 2:
                vmin, vmax = 0.0, 1.0
                label = 'Refletância (canal visível)'
                cmap = cm.gray

            elif band == 6:
                vmin, vmax = 0.0, 0.5
                label = 'Radiância (SWIR - 2.2 µm)'
                cmap = cm.cubehelix

            elif band == 7:
                data = data - 273.15
                vmin, vmax = 0.0, 60
                label = 'Radiância (SWIR - 3.9 µm)'
                cmap = cm.plasma

            elif band == 13:
                data = data - 273.15
                vmin, vmax = -80, 30
                label = 'Temperatura de brilho (°C)'
                cmap = loadCPT('IR4AVHRR6.cpt')

            colormap = cm.colors.LinearSegmentedColormap('cpt', cmap) if isinstance(cmap, dict) else cmap

            # Figura
            plt.figure(figsize=(10,10))
            ax = plt.axes(projection=ccrs.PlateCarree())
            img_extent = [extent[0], extent[2], extent[1], extent[3]]
            img = ax.imshow(data, origin='upper', extent=img_extent, vmin=vmin, vmax=vmax, cmap=colormap)

            ax.coastlines(resolution='10m', color='white', linewidth=0.8)
            ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
            gl = ax.gridlines(draw_labels=True, xlocs=np.arange(-180,180,5), ylocs=np.arange(-90,90,5),
                              linestyle='--', linewidth=0.25, color='gray', alpha=1.0)
            gl.top_labels = gl.right_labels = False

            shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
            ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

            # Tempo
            date = datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            date_str = date.strftime('%Y-%m-%d %H:%M UTC')
            title_band = f"GOES-16 {band_titles[band]}"

            plt.colorbar(img, label=label, extend='both', orientation='horizontal', pad=0.05, fraction=0.05)
            plt.title(f'{title_band}\n{date_str}', fontweight='bold', fontsize=10, loc='left')
            plt.title(f'Reg.: {extent}', fontsize=10, loc='right')

            out_file = f'{output_dir}/GOES16_B{band}_{yyyymmddhhmn}.png'
            plt.savefig(out_file, bbox_inches='tight', pad_inches=0.1, dpi=300)
            plt.close()
            print(f'[✓] Imagem salva: {out_file}')
        except Exception as e:
            print(f'[!] Erro para {yyyymmdd} B{band}: {e}')
    current_date += timedelta(days=1)



In [ ]:
#@title Composto RGB - Com Estrela
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from utilities import download_CMI, remap
import os

# Parâmetros
input_dir = "./samples"
output_dir = "./output_rgb"
os.makedirs(output_dir, exist_ok=True)

extent = [-90.0, -60.0, -30.0, 15.0]  # América do Sul
bands_rgb = [7, 6, 2]  # R, G, B
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
hour = 18

# Coordenadas das estações e cores das estrelas
estacoes = {
    'São_Paulo': {'coords': (-23.5505, -46.6333), 'cor': 'white'},
    'Cuiaba-Miranda': {'coords': (-15.73091, -56.07086), 'cor': 'darkred'},
    'Alta-Floresta': {'coords': (-9.908354, -56.064393), 'cor': 'darkgreen'}
}

def normalize(array, vmin, vmax, gamma=1):
    array = np.clip(array, vmin, vmax)
    array = ((array - vmin) / (vmax - vmin)) ** (1/gamma)
    return array

# Função para converter coordenadas geográficas para pixels
def latlon_to_pixel(lat, lon, extent, img_shape):
    lon_min, lat_min, lon_max, lat_max = extent
    x = (lon - lon_min) / (lon_max - lon_min) * img_shape[1]
    y = (lat_max - lat) / (lat_max - lat_min) * img_shape[0]
    return x, y

# Loop por data
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
    print(f'\n>>> {yyyymmdd} - Composição RGB (7-6-2)')

    try:
        rgb_arrays = []
        for band in bands_rgb:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1:
                raise Exception(f"Arquivo não disponível para o canal {band}")

            path = os.path.join(input_dir, file_name + '.nc')
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            # Normalização e ajustes por banda
            if band == 7:
                arr = normalize(data - 273.15, 0, 60, gamma=0.4)  # R (temperatura de brilho)
            elif band == 6:
                arr = normalize(data, 0, 1, gamma=1)  # G (detecção de partículas finas e vapor)
            elif band == 2:
                arr = normalize(data, 0, 0.75, gamma=1)  # B (visível)

            rgb_arrays.append(arr)

        # Empilhamento RGB (invertido para BGR → RGB)
        RGB = np.stack(rgb_arrays[::-1], axis=2)

        # Máscara para valores fora do globo
        mask = (RGB == RGB[0, 0, :]).all(axis=2)
        RGB[mask] = np.nan

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(RGB, origin='upper')
        ax.set_title(f"GOES-16 RGB 7-6-2 - {yyyymmddhhmn}", fontsize=12, color='white')
        ax.axis('off')
        fig.patch.set_facecolor('black')

        # Adicionando as estrelas e os nomes
        img_shape = RGB.shape[:2]  # Dimensões da imagem (altura, largura)
        for nome, info in estacoes.items():
            lat, lon = info['coords']
            cor = info['cor']
            # Converter coordenadas geográficas para pixels
            x, y = latlon_to_pixel(lat, lon, extent, img_shape)
            # Estrela colorida com contorno preto
            ax.plot(x, y, marker='*', color=cor, markersize=15,
                    markeredgecolor='black', markeredgewidth=1.5)
            # Nome sempre branco
            ax.text(x + 20, y - 20, nome.replace('-', ' '), color='white',
                    fontsize=10, weight='bold')

        # Salvar imagem
        out_path = os.path.join(output_dir, f"FTP_RGB_{yyyymmddhhmn}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0, facecolor='black', dpi=300)
        plt.close()
        print(f'[✓] Imagem salva: {out_path}')

    except Exception as e:
        print("[Erro]", e)

    current_date += timedelta(days=1)

In [ ]:
#@title Composto RGB
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from utilities import download_CMI, remap
import os

# Parâmetros
input_dir = "./samples"
output_dir = "./output_rgb"
os.makedirs(output_dir, exist_ok=True)

extent = [-90.0, -60.0, -30.0, 15.0]  # América do Sul
bands_rgb = [7, 6, 2]  # R, G, B
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
hour = 18

def normalize(array, vmin, vmax, gamma=1):
    array = np.clip(array, vmin, vmax)
    array = ((array - vmin) / (vmax - vmin)) ** (1/gamma)
    return array

# Loop por data
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
    print(f'\n>>> {yyyymmdd} - Composição RGB (7-6-2)')

    try:
        rgb_arrays = []
        for band in bands_rgb:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1:
                raise Exception(f"Arquivo não disponível para o canal {band}")

            path = os.path.join(input_dir, file_name + '.nc')
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            # Normalização e ajustes por banda
            if band == 7:
                arr = normalize(data - 273.15, 0, 60, gamma=0.4)  # R (temperatura de brilho)
            elif band == 6:
                arr = normalize(data, 0, 1, gamma=1)  # G (detecção de partículas finas e vapor)
            elif band == 2:
                arr = normalize(data, 0, 0.75, gamma=1)  # B (visível)

            rgb_arrays.append(arr)

        # Empilhamento RGB (invertido para BGR → RGB)
        RGB = np.stack(rgb_arrays[::-1], axis=2)

        # Máscara para valores fora do globo
        mask = (RGB == RGB[0, 0, :]).all(axis=2)
        RGB[mask] = np.nan

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(RGB, origin='upper')
        ax.set_title(f"GOES-16 RGB 7-6-2 - {yyyymmddhhmn}", fontsize=12, color='white')
        ax.axis('off')
        fig.patch.set_facecolor('black')

        # Salvar imagem
        out_path = os.path.join(output_dir, f"FTP_RGB_{yyyymmddhhmn}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0, facecolor='black')
        plt.close()

    except Exception as e:
        print("[Erro]", e)

    current_date += timedelta(days=1)


In [ ]:
#@title Composto RGB + HOTSPOT - Com Estrela
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from utilities import download_CMI, remap
import os

# Diretórios
input_dir = "./samples"
output_dir = "./output_rgb"
os.makedirs(output_dir, exist_ok=True)

# Extensão geográfica e canais
extent = [-90.0, -60.0, -30.0, 15.0]
bands_rgb = [7, 6, 2]  # R: C07, G: C06, B: C02
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
hour = 18

# Coordenadas das estações e cores das estrelas
estacoes = {
    'São_Paulo': {'coords': (-23.5505, -46.6333), 'cor': 'white'},
    'Cuiaba-Miranda': {'coords': (-15.73091, -56.07086), 'cor': 'darkred'},
    'Alta-Floresta': {'coords': (-9.908354, -56.064393), 'cor': 'darkgreen'}
}

def scale_channel(channel, vmin, vmax):
    return np.clip((channel - vmin) / (vmax - vmin), 0, 1)

def apply_gamma(channel, gamma):
    return np.power(channel, gamma)

# Função para converter coordenadas geográficas para pixels
def latlon_to_pixel(lat, lon, extent, img_shape):
    lon_min, lat_min, lon_max, lat_max = extent
    x = (lon - lon_min) / (lon_max - lon_min) * img_shape[1]
    y = (lat_max - lat) / (lat_max - lat_min) * img_shape[0]
    return x, y

# Loop por data
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
    print(f'\n>>> {yyyymmdd} - Composição RGB (7-6-2)')

    try:
        rgb_arrays = []
        hotspot_coords = None

        for band in bands_rgb:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1:
                raise Exception(f"Arquivo indisponível para o canal {band}")

            path = os.path.join(input_dir, file_name + '.nc')
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            if band == 7:
                temp = data  # temperatura em Kelvin
                c07_scaled = scale_channel(temp, 290, 330)  # Foco quente
                c07_scaled = apply_gamma(c07_scaled, 1.0)

                # Identifica hotspots (>330 K)
                hotspot_mask = temp > 333
                hotspot_coords = np.where(hotspot_mask)
                arr = c07_scaled

            elif band == 6:
                arr = scale_channel(data, 0, 0.5)
                arr = apply_gamma(arr, 0.6)

            elif band == 2:
                arr = scale_channel(data, 0, 0.7)
                arr = apply_gamma(arr, 0.6)

            rgb_arrays.append(arr)

        # Empilha RGB
        RGB = np.stack([rgb_arrays[0], rgb_arrays[1], rgb_arrays[2]], axis=2)  # R, G, B

        # Máscara para fora do globo
        mask = (RGB == RGB[0, 0, :]).all(axis=2)
        RGB[mask] = np.nan

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(RGB, origin='upper')
        ax.set_title(f"GOES-16 RGB 7-6-2 c/ Hotspots - {yyyymmddhhmn}", color='white')
        ax.axis('off')

        # Scatter dos hotspots
        if hotspot_coords is not None:
            y, x = hotspot_coords
            ax.scatter(x, y, s=2, c='red', label='Hotspots')

        # Adicionando as estrelas e os nomes
        img_shape = RGB.shape[:2]  # Dimensões da imagem (altura, largura)
        for nome, info in estacoes.items():
            lat, lon = info['coords']
            cor = info['cor']
            # Converter coordenadas geográficas para pixels
            x, y = latlon_to_pixel(lat, lon, extent, img_shape)
            # Estrela colorida com contorno preto
            ax.plot(x, y, marker='*', color=cor, markersize=15,
                    markeredgecolor='black', markeredgewidth=1.5)
            # Nome sempre branco
            ax.text(x + 20, y - 20, nome.replace('-', ' '), color='white',
                    fontsize=10, weight='bold')

        # Salvar
        out_path = os.path.join(output_dir, f"FTP_RGB_HP_Vento_{yyyymmddhhmn}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0, facecolor='black', dpi=300)
        plt.close()
        print(f'[✓] Imagem salva: {out_path}')

    except Exception as e:
        print("[Erro]", e)

    current_date += timedelta(days=1)

In [ ]:
#@title Composto RGB + Hotspots
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from utilities import download_CMI, remap
import os

# Diretórios
input_dir = "./samples"
output_dir = "./output_rgb"
os.makedirs(output_dir, exist_ok=True)

# Extensão geográfica e canais
extent = [-90.0, -60.0, -30.0, 15.0]
bands_rgb = [7, 6, 2]  # R: C07, G: C06, B: C02
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
hour = 18

def scale_channel(channel, vmin, vmax):
    return np.clip((channel - vmin) / (vmax - vmin), 0, 1)

def apply_gamma(channel, gamma):
    return np.power(channel, gamma)

# Loop por data
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
    print(f'\n>>> {yyyymmdd} - Composi\u00e7\u00e3o RGB (7-6-2)')

    try:
        rgb_arrays = []
        hotspot_coords = None

        for band in bands_rgb:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1:
                raise Exception(f"Arquivo indispon\u00edvel para o canal {band}")

            path = os.path.join(input_dir, file_name + '.nc')
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            if band == 7:
                temp = data  # temperatura em Kelvin
                c07_scaled = scale_channel(temp, 290, 330)  # Foco quente
                c07_scaled = apply_gamma(c07_scaled, 1.0)

                # Identifica hotspots (>330 K)
                hotspot_mask = temp > 333
                hotspot_coords = np.where(hotspot_mask)
                arr = c07_scaled

            elif band == 6:
                arr = scale_channel(data, 0, 0.5)
                arr = apply_gamma(arr, 0.6)

            elif band == 2:
                arr = scale_channel(data, 0, 0.7)
                arr = apply_gamma(arr, 0.6)

            rgb_arrays.append(arr)

        # Empilha RGB
        RGB = np.stack([rgb_arrays[0], rgb_arrays[1], rgb_arrays[2]], axis=2)  # R, G, B

        # Máscara para fora do globo
        mask = (RGB == RGB[0, 0, :]).all(axis=2)
        RGB[mask] = np.nan

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(RGB, origin='upper')
        ax.set_title(f"GOES-16 RGB 7-6-2 c/ Hotspots - {yyyymmddhhmn}", color='white')
        ax.axis('off')

        # Scatter dos hotspots
        if hotspot_coords is not None:
            y, x = hotspot_coords
            ax.scatter(x, y, s=2, c='red', label='Hotspots')

        # Salvar
        out_path = os.path.join(output_dir, f"FTP_RGB_HP_{yyyymmddhhmn}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0, facecolor='black')
        plt.close()

    except Exception as e:
        print("[Erro]", e)

    current_date += timedelta(days=1)

In [ ]:
#@title Composto RGB (7-6-2) sem Hotspots
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from utilities import download_CMI, remap
import os

# Diretórios
input_dir = "./samples"
output_dir = "./output_rgb"
os.makedirs(output_dir, exist_ok=True)

# Extensão geográfica e canais
extent = [-90.0, -60.0, -30.0, 15.0]
bands_rgb = [7, 6, 2]  # R: C07, G: C06, B: C02
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
hour = 18

def scale_channel(channel, vmin, vmax):
    return np.clip((channel - vmin) / (vmax - vmin), 0, 1)

def apply_gamma(channel, gamma):
    return np.power(channel, gamma)

# Loop por data
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
    print(f'\n>>> {yyyymmdd} - Composição RGB (7-6-2)')

    try:
        rgb_arrays = []

        for band in bands_rgb:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1:
                raise Exception(f"Arquivo indisponível para o canal {band}")

            path = os.path.join(input_dir, file_name + '.nc')
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            if band == 7:
                temp = data  # temperatura em Kelvin
                c07_scaled = scale_channel(temp, 290, 330)  # Foco quente
                c07_scaled = apply_gamma(c07_scaled, 1.0)
                arr = c07_scaled

            elif band == 6:
                arr = scale_channel(data, 0, 0.5)
                arr = apply_gamma(arr, 0.6)

            elif band == 2:
                arr = scale_channel(data, 0, 0.7)
                arr = apply_gamma(arr, 0.6)

            rgb_arrays.append(arr)

        # Empilha RGB
        RGB = np.stack([rgb_arrays[0], rgb_arrays[1], rgb_arrays[2]], axis=2)  # R, G, B

        # Máscara para fora do globo
        mask = (RGB == RGB[0, 0, :]).all(axis=2)
        RGB[mask] = np.nan

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(RGB, origin='upper')
        ax.set_title(f"GOES-16 RGB 7-6-2 - {yyyymmddhhmn}", color='white')
        ax.axis('off')

        # Salvar
        out_path = os.path.join(output_dir, f"FTP_RGB_{yyyymmddhhmn}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0, facecolor='black')
        plt.close()

    except Exception as e:
        print("[Erro]", e)

    current_date += timedelta(days=1)



In [ ]:
#@title Composto RGB (7-6-2) sem Hotspots - Com Estrela
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from utilities import download_CMI, remap
import os

# Diretórios
input_dir = "./samples"
output_dir = "./output_rgb"
os.makedirs(output_dir, exist_ok=True)

# Extensão geográfica e canais
extent = [-90.0, -60.0, -30.0, 15.0]
bands_rgb = [7, 6, 2]  # R: C07, G: C06, B: C02
start_date = datetime(2024, 8, 20)
end_date = datetime(2024, 9, 10)
hour = 18

# Coordenadas das estações e cores das estrelas
estacoes = {
    'São_Paulo': {'coords': (-23.5505, -46.6333), 'cor': 'white'},
    'Cuiaba-Miranda': {'coords': (-15.73091, -56.07086), 'cor': 'darkred'},
    'Alta-Floresta': {'coords': (-9.908354, -56.064393), 'cor': 'darkgreen'}
}

def scale_channel(channel, vmin, vmax):
    return np.clip((channel - vmin) / (vmax - vmin), 0, 1)

def apply_gamma(channel, gamma):
    return np.power(channel, gamma)

# Função para converter coordenadas geográficas para pixels
def latlon_to_pixel(lat, lon, extent, img_shape):
    lon_min, lat_min, lon_max, lat_max = extent
    x = (lon - lon_min) / (lon_max - lon_min) * img_shape[1]
    y = (lat_max - lat) / (lat_max - lat_min) * img_shape[0]
    return x, y

# Loop por data
current_date = start_date
while current_date <= end_date:
    yyyymmdd = current_date.strftime('%Y%m%d')
    yyyymmddhhmn = yyyymmdd + f'{hour:02}00'
    print(f'\n>>> {yyyymmdd} - Composição RGB (7-6-2)')

    try:
        rgb_arrays = []

        for band in bands_rgb:
            file_name = download_CMI(yyyymmddhhmn, band, input_dir)
            if file_name == -1:
                raise Exception(f"Arquivo indisponível para o canal {band}")

            path = os.path.join(input_dir, file_name + '.nc')
            grid = remap(path, 'CMI', extent, 2)
            data = grid.ReadAsArray()

            if band == 7:
                temp = data  # temperatura em Kelvin
                c07_scaled = scale_channel(temp, 290, 330)  # Foco quente
                c07_scaled = apply_gamma(c07_scaled, 1.0)
                arr = c07_scaled

            elif band == 6:
                arr = scale_channel(data, 0, 0.5)
                arr = apply_gamma(arr, 0.6)

            elif band == 2:
                arr = scale_channel(data, 0, 0.7)
                arr = apply_gamma(arr, 0.6)

            rgb_arrays.append(arr)

        # Empilha RGB
        RGB = np.stack([rgb_arrays[0], rgb_arrays[1], rgb_arrays[2]], axis=2)  # R, G, B

        # Máscara para fora do globo
        mask = (RGB == RGB[0, 0, :]).all(axis=2)
        RGB[mask] = np.nan

        # Plot
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(RGB, origin='upper')
        ax.set_title(f"GOES-16 RGB 7-6-2 - {yyyymmddhhmn}", color='white')
        ax.axis('off')

        # Adicionando as estrelas e os nomes
        img_shape = RGB.shape[:2]  # Dimensões da imagem (altura, largura)
        for nome, info in estacoes.items():
            lat, lon = info['coords']
            cor = info['cor']
            # Converter coordenadas geográficas para pixels
            x, y = latlon_to_pixel(lat, lon, extent, img_shape)
            # Estrela colorida com contorno preto
            ax.plot(x, y, marker='*', color=cor, markersize=15,
                    markeredgecolor='black', markeredgewidth=1.5)
            # Nome sempre branco
            ax.text(x + 20, y - 20, nome.replace('-', ' '), color='white',
                    fontsize=10, weight='bold')

        # Salvar
        out_path = os.path.join(output_dir, f"FTP_RGB_1_{yyyymmddhhmn}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0, facecolor='black', dpi=300)
        plt.close()
        print(f'[✓] Imagem salva: {out_path}')

    except Exception as e:
        print("[Erro]", e)

    current_date += timedelta(days=1)

In [ ]:
#@title Baixando Imagens no PC

import shutil

# Compacta a pasta 'output_rgb' em um arquivo ZIP
shutil.make_archive("rgbs_imagens", 'zip', "output_rgb")

from google.colab import files

# Faz o download do arquivo compactado
files.download("rgbs_imagens.zip")

